In [2]:
l = 1000
with open("../data/tables.json", "r") as infile, open(f"../data/tables_{l}.json", "w") as outfile:
    for i, line in enumerate(infile):
        if i >= l:
            break
        outfile.write(line)

In [3]:
import pandas as pd
import numpy as np 
import torch 
import matplotlib.pyplot as plt 
import collections
import nltk 
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import string
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk import ngrams
import copy
from collections import defaultdict
from functools import cache
import cProfile
import pstats
import time
from tabulate import tabulate
from copy import deepcopy

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [4]:
# Utility functions
def get_clock_time_in_milli_sec():
    return int(time.time() * 1000)

def print_time(milli_sec):
    hours, remainder = divmod(milli_sec, 1000 * 60 * 60)
    minutes, remainder = divmod(remainder, 1000 * 60)
    seconds, milli_seconds = divmod(remainder, 1000)
    components = []
    if hours:
        components.append(f"{hours}h")
    if minutes:
        components.append(f"{minutes}m")
    if seconds:
        components.append(f"{seconds}s")
    if milli_seconds:
        components.append(f"{milli_seconds}ms")
    print(f"{milli_sec}[{':'.join(components)}]")


In [5]:
def create_table_list(dfs):
    return [[[cell.get('text', '') for cell in row] for row in table_data] for table_data in dfs['tableData']]

In [6]:
@cache
def process_words(text:str)->list: 
    sentence = text.lower().translate(str.maketrans("", "", string.punctuation))
    words = word_tokenize(sentence)
    return words 

In [7]:
ps = PorterStemmer()

In [8]:
@cache
def stem_cached(word):
    return ps.stem(word)

In [9]:
def create_projections(table_list,
                       start_table_index,
                       projections,
                       stemmer, 
                       stop_words: list = None, 
                       min_word_len = 0, 
                       create_ngrams:bool = False, 
                       ngrams_size:int = 2
                       ) -> dict: 

    ps = stemmer
    
    if stop_words: 
        stemmed_stopwords = {ps.stem_cached(w) for w in stop_words}
    else:
        stemmed_stopwords = set()

    for table_index, table in enumerate(table_list): 
        for row_index, row in enumerate(table): 
            for column_index, cell in enumerate(row): 

                words = process_words(cell)

                if create_ngrams: 
                    ngram_stem_set = set()

                for word in words: 
                    stemmed_word = stem_cached(word)
                        
                    if stemmed_word not in stemmed_stopwords and len(stemmed_word) >= min_word_len: 

                        projections.setdefault(stemmed_word, set()).add((table_index, row_index, column_index))

                        if create_ngrams: 
                            ngram_stem_set.update(stemmed_word)
                
                if create_ngrams: 
                    created_ngrams = ngrams(ngram_stem_set, ngrams_size)
                    for created_ngram in created_ngrams: 
                        projections.setdefault(created_ngram, set()).add((table_index, row_index, column_index))
    
    return projections

In [10]:
def create_tables_df(table_chunk, start_table_index, tables_df): 

    new_tables_df = pd.DataFrame({
        "id": range(start_table_index, start_table_index + len(table_chunk)),
        "pgTitle": table_chunk.get('pgTitle', ''),
        "sectionTitle": table_chunk.get('sectionTitle', ''),
        "tableCaption": table_chunk.get('tableCaption', ''),
        "prior": [0.5] * len(table_chunk),
        "score": [None] * len(table_chunk)
    })

    if not tables_df.empty:
        tables_df = pd.concat([tables_df, new_tables_df], ignore_index=True)
    else:
        tables_df = new_tables_df

    return tables_df

In [11]:
def cleaner(value, create_ngrams:bool = False, ngram_size:int = 2) -> list: ####Umbenennen 
    """
    Returns the tokenized and stemed version of a Value
    """
    words = process_words(value)

    stemmed_words = [stem_cached(word) for word in words]

    if create_ngrams: 
        created_ngrams = ngrams(stemmed_words, ngram_size)
        stemmed_words.extend(list(created_ngrams))
        
    return stemmed_words

In [12]:
def indexing(cleaned_values:list, projections:dict)->dict: 
    """
    Return a dict of all the Examples found in the projections
    In: 
        Cleaned_Values: A List of all the Stemped Versions of one Example given
        Projections: A Dict of Projections of all given Tables
    Out: 
        Index_Dict: A dict of all the positions where the Example was found
                    Form: Key: (Table_ID, Row_ID) -> Value: (Col_ID)
    """
    index_dict = dict() 
    
    for value in cleaned_values: 
        value_index = projections.get(value, None)

        if value_index: 
            for index_pair in value_index: 
                table_id, row_id, col_id = index_pair 
                index_dict.setdefault((table_id, row_id), set()).add((col_id))
    
    return index_dict

In [13]:
def compareit(dictx:dict, dicty:dict, sub_key_mode:bool = False)-> list: 
    """
    Compairs two Dicts with eachother and return a Dict of Intersections between Dicts 
    In: 
        Dictx: Dict of the Form Key: (Table_ID, Row_ID) -> Value: (Row_ID(s))
        Dicty: Dict of the Form Key: (Table_ID, Row_ID) -> Value: (Row_ID(s))
        Sub_Key_Mode: Bool that assures that the output dict is in a Form that another instance of 
                      Compareit could use it
    Out: 
        Possible_Tables_Dict/ Subkey_Dict: A Dict of all Intersections between Dictx and Dicty
                                           In the Form of Dictx/ Dicty if Sub_Key_Mode == True 
    """

    if not sub_key_mode: 
        possible_tables_dict = dict()
    else: 
        subkey_dict = dict()
    intersecting_keys = set(dictx) & set(dicty)
    for key in intersecting_keys: 
        table_id, row_id = key
        for x_col_id in dictx[key]: 
            for y_col_id in dicty[key]: 
                if not sub_key_mode: 
                    possible_tables_dict.setdefault(table_id, set()).add((row_id, (x_col_id, y_col_id)))
                    
                else: 
                    subkey_dict.setdefault(key, set()).add((x_col_id, y_col_id))
    if not sub_key_mode: 
        return possible_tables_dict
    else: 
        return subkey_dict

In [14]:
def querry_thunel(example_pairs:list, projections:dict, tau:int=1):
    """
    Perfoms the complete Querrying Web Tables Operation. 
    In: 
        Example_Pairs: A List of the Semmantic Mapping 
        Projections: A Dict of Projections of all given Tables
        Tau: An Int which indicates how many Example_pairs must be in a Table to count as relevant
    Out: 
        Tables: A Dict of all relevant Tables 
                Form: Key: Table_ID -> Value: (Row_ID, (XColumn_ID, YColumn_ID))
    """
    
    if len(example_pairs) < tau: 
        raise ValueError(f"The Cardinality of given Example_Pairs {len(example_pairs)} must be greater or qual then Tau: {tau}!")
    

    possible_tables = dict()

    for tup in example_pairs: 
        list_of_keys = list()
        for val in tup: 
            
            if isinstance(val, tuple): 
                list_of_subkeys = list()
                for sub_key in val: 
                    cleaned_sub_key = cleaner(sub_key)
                    index_of_sub_key = indexing(cleaned_sub_key, projections)
                    list_of_subkeys.append(index_of_sub_key)
                key_val = compareit(*list_of_subkeys, sub_key_mode=True)
                
            
            else: 
                cleaned_key = cleaner(val)
                index_of_key = indexing(cleaned_key, projections)
                key_val = index_of_key
            
            list_of_keys.append(key_val)
        
        

        compared_things = compareit(*list_of_keys)
        possible_tables.update(compared_things)

    tables = dict()
    for key in possible_tables: 
        if len(possible_tables[key]) >= tau: 
            tables[key] = possible_tables[key]

    return tables 

In [15]:
def iter_json(path, csize):
    iter_chunk = pd.read_json(path, lines=True, chunksize=csize)
    for chunk in iter_chunk:
        yield chunk

In [23]:
# work with getter and setter and global / environment variables

def update_table_scores(query_answers, query_tables_ids, tables_df, answer_scores, table_scores):
    alpha = 0.99
    
    for query_table_id in query_tables_ids:
        good = 0
        bad = 0
        covered_query_answers_x = set()

        table_answers = set(zip(
                query_answers.loc[query_answers['table_id'] == query_table_id, 'answer_x'],
                query_answers.loc[query_answers['table_id'] == query_table_id, 'answer_y']
                ))

        for table_answer in table_answers:
            covered_query_answers_x.add(table_answer[0])
            table_answer_score = answer_scores[table_answer]
            if (table_answer_score == max([score for (x, _), score in answer_scores.items() if x == table_answer[0]])):
                good += table_answer_score
            else:
                bad += 1

        unseen_x = 0
        for query_answer in zip(query_answers['answer_x'], query_answers['answer_y']):
            if not query_answer[0] in covered_query_answers_x:
                unseen_x += max([score for answer, score in answer_scores.items() if answer == query_answer])

        table_prior = tables_df.loc[tables_df['id'] == query_table_id, 'prior'].iloc[0]

        table_score = alpha * ((table_prior * good) / (table_prior * good + (1-table_prior) * (bad + unseen_x)))
        table_scores[query_table_id] = table_score

    return table_scores


In [17]:
def update_answer_scores(query_answers, query_answers_set, answer_scores, table_scores):

    query_answer_xs = {x for x, _ in query_answers_set}
    
    for query_answers_x in query_answer_xs:
        
        x_answers = {answer for answer in query_answers_set if answer[0] == query_answers_x}

        score_of_none = 1
        
        for table_id in query_answers.loc[query_answers['answer_x'] == query_answers_x, 'table_id']:
            score_of_none *= (1-table_scores[table_id])
            for x_answer in x_answers:
                answer_score = 1

                if x_answer in set(zip(
                    query_answers.loc[query_answers['table_id'] == table_id, 'answer_x'],
                    query_answers.loc[query_answers['table_id'] == table_id, 'answer_y']
                )):
                    answer_score *= table_scores[table_id]
                else:
                    answer_score *= (1-table_scores[table_id])

        for x_answer in x_answers:
            answer_scores[x_answer] = answer_scores[x_answer] / (score_of_none + np.sum(np.array([score for answer, score in answer_scores if answer in x_answers])))

    return answer_scores

In [18]:
# query_answers format: dataframe(answer_x, answer_y, table_id)

def expectation_maximization(query_answers, epsilon, tables_df):
    query_tables_ids = set(query_answers['table_id'])
    query_answers_set = set(zip(query_answers['answer_x'], query_answers['answer_y']))

    # which values for initilization?
    answer_scores = {answer: 1.0 for answer in query_answers_set}
    table_scores = {table_id: 1.0 for table_id in query_tables_ids}

    delta_score = np.inf

    old_answer_scores = dict()

    while delta_score > epsilon:

        # add line 6-15?

        old_answer_scores = deepcopy(answer_scores)

        table_scores = update_table_scores(query_answers, query_tables_ids, tables_df, answer_scores, table_scores)
        answer_scores = update_answer_scores(query_answers, query_answers_set, answer_scores, table_scores)

        delta_score = np.sum(np.abs(
                np.array([answer_scores[answer] for answer in query_answers_set]) -
                np.array([old_answer_scores[answer] for answer in query_answers_set])
            ))

    print(table_scores)

    return answer_scores

In [19]:
rows = [
    ('FCB', 'FC Barcelona'),
    ('FCB', 'FC Bayern München'),
    ('FCB', 'Football Club Barcelona'),
    ('NYC', 'New York City'),
    ('NYC', 'NY City'),
    ('USA', 'United States'),
    ('USA', 'U.S.A.'),
    ('USA', 'United States of America'),
    ('IBM', 'International Business Machines'),
    ('IBM', 'IBM Corp.'),
    ('MIT', 'Massachusetts Institute of Technology'),
    ('MIT', 'M.I.T.'),
    ('NVIDIA', 'Nvidia Corporation'),
    ('NVIDIA', 'NVIDIA Inc.'),
    ('UCLA', 'University of California, Los Angeles'),
    ('UCLA', 'U.C.L.A.'),
    ('Dept.', 'Department'),
    ('Dept.', 'Dept'),
    ('Google', 'Google LLC'),
    ('Google', 'Google Inc.'),
]

query_answers = pd.DataFrame(rows, columns=['answer_x', 'answer_y'])
query_answers['table_id'] = list(range(10)) * 2

In [20]:
start_table_index = 0

csize = 10
path = "../data/tables_10.json"

tables_df = pd.DataFrame()

for table_chunk in iter_json(path, csize):
    tables_df = create_tables_df(table_chunk, start_table_index, tables_df)

In [21]:
print(tabulate(tables_df, headers='keys', tablefmt='psql'))
print(tabulate(query_answers, headers='keys', tablefmt='psql'))

+----+------+-------------------------------------------------------+---------------------------+---------------------------+---------+---------+
|    |   id | pgTitle                                               | sectionTitle              | tableCaption              |   prior | score   |
|----+------+-------------------------------------------------------+---------------------------+---------------------------+---------+---------|
|  0 |    0 | Mid Antrim (Northern Ireland Parliament constituency) | Members of Parliament     | Members of Parliament     |     0.5 |         |
|  1 |    1 | Römer (crater)                                        | Satellite craters         | Satellite craters         |     0.5 |         |
|  2 |    2 | Whispermoon                                           |                           | Track listing             |     0.5 |         |
|  3 |    3 | Khalsa Diwan Society Vancouver                        | First executive committee | First executive committee 

In [24]:
expectation_maximization(query_answers, 0.1, tables_df)

{0: nan, 1: nan, 2: nan, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0}


C:\Users\User\AppData\Local\Temp\ipykernel_8212\3407514113.py:25: RuntimeWarning: overflow encountered in scalar divide
  answer_scores[x_answer] = answer_scores[x_answer] / (score_of_none + np.sum(np.array([score for answer, score in answer_scores if answer in x_answers])))
C:\Users\User\AppData\Local\Temp\ipykernel_8212\823063290.py:31: RuntimeWarning: invalid value encountered in scalar divide
  table_score = alpha * ((table_prior * good) / (table_prior * good + (1-table_prior) * (bad + unseen_x)))


{('IBM', 'International Business Machines'): 3.9144124360393073,
 ('FCB', 'FC Bayern München'): nan,
 ('MIT', 'M.I.T.'): nan,
 ('Dept.', 'Dept'): 19715424296.18966,
 ('Google', 'Google LLC'): 3.9144124360393073,
 ('USA', 'U.S.A.'): 1816626212302610.5,
 ('USA', 'United States'): 1816626212302610.5,
 ('Dept.', 'Department'): 19715424296.18966,
 ('UCLA', 'University of California, Los Angeles'): 196177.20327473822,
 ('USA', 'United States of America'): 1816626212302610.5,
 ('FCB', 'FC Barcelona'): nan,
 ('NYC', 'New York City'): 4.532921321014106,
 ('MIT', 'Massachusetts Institute of Technology'): nan,
 ('NVIDIA', 'Nvidia Corporation'): nan,
 ('UCLA', 'U.C.L.A.'): 196177.20327473822,
 ('NVIDIA', 'NVIDIA Inc.'): nan,
 ('IBM', 'IBM Corp.'): 3.9144124360393073,
 ('Google', 'Google Inc.'): 3.9144124360393073,
 ('FCB', 'Football Club Barcelona'): nan,
 ('NYC', 'NY City'): 4.532921321014106}

In [15]:
if __name__ == "__main__":

    csize = 10
    path = "../data/tables_10.json"

    start_table_index = 0

    projections = dict()
    tables_df = pd.DataFrame()

    c = get_clock_time_in_milli_sec()

    profiler = cProfile.Profile()
    profiler.enable()

    for table_chunk in iter_json(path, csize):
        
        table_list = create_table_list(table_chunk)

        projections = create_projections(table_list, start_table_index, projections, ps, create_ngrams=True, min_word_len=3)

        tables_df = create_tables_df(table_chunk, start_table_index, tables_df)

        start_table_index += csize

    print(tabulate(tables_df, headers='keys', tablefmt='psql'))

    c = get_clock_time_in_milli_sec() - c

    print("Time=", end="")
    print_time(c)
    print()

    profiler.disable()
    stats = pstats.Stats(profiler)
    stats.strip_dirs()
    stats.sort_stats("time")
    stats.print_stats()

    indexing_examples = [('1929', 'Robert Crawford'), ('1933', 'Robert Crawford')]
    # multikey_indexing_example = [(('1929', 'Robert Crawford'), 'Ulster Unionist')]
    tau = 1
    final_list_single_key_dict = defaultdict(dict)

    for indexing_example in indexing_examples:

        dict_test = querry_thunel([indexing_example], projections, tau)
        final_list_single_key_dict[indexing_example].update(dict_test)
    
    # final_list_muli_key = querry_thunel(multikey_indexing_example, projections, tau)

    # expectation_maximization(final_list_single_key_dict, projections)

    print(final_list_single_key_dict)
    # print(final_list_muli_key)

+----+------+-------------------------------------------------------+---------------------------+---------------------------+---------+---------+
|    |   id | pgTitle                                               | sectionTitle              | tableCaption              |   prior | score   |
|----+------+-------------------------------------------------------+---------------------------+---------------------------+---------+---------|
|  0 |    0 | Mid Antrim (Northern Ireland Parliament constituency) | Members of Parliament     | Members of Parliament     |     0.5 |         |
|  1 |    1 | Römer (crater)                                        | Satellite craters         | Satellite craters         |     0.5 |         |
|  2 |    2 | Whispermoon                                           |                           | Track listing             |     0.5 |         |
|  3 |    3 | Khalsa Diwan Society Vancouver                        | First executive committee | First executive committee 

In [33]:
# --- III-C (1/4): purge index entries that point outside current table_list ---

def purge_bad_postings(projections, table_list):
    """
    Remove any (tid, rid, cid) that point outside table_list bounds.
    Returns number of removed postings.
    """
    removed = 0
    max_tid = len(table_list) - 1
    for token in list(projections.keys()):
        postings = projections[token]
        good = set()
        for (tid, rid, cid) in postings:
            if not (0 <= tid <= max_tid):
                continue
            tbl = table_list[tid]
            if not (0 <= rid < len(tbl)):
                continue
            row = tbl[rid]
            if not (0 <= cid < len(row)):
                continue
            good.add((tid, rid, cid))
        if good:
            removed += (len(postings) - len(good))
            projections[token] = good
        else:
            removed += len(postings)
            del projections[token]
    return removed

# Run once to sanitize the current projections
_ = purge_bad_postings(projections, table_list)

In [34]:
# --- III-C (2/4): guarded lookups used only by part C ---

from collections import defaultdict

def _value_hits(projections, value, create_ngrams=False, ngram_size=2):
    toks = cleaner(value, create_ngrams, ngram_size)
    postings = set()
    for t in toks:
        postings |= projections.get(t, set())
    return postings

def rows_containing_value_in_col(table_list, table_id, col_id, value, create_ngrams=False, ngram_size=2):
    if not (0 <= table_id < len(table_list)):
        return set()
    toks = set(cleaner(value, create_ngrams, ngram_size))
    if not toks:
        return set()
    rows = set()
    tbl = table_list[table_id]
    for r, row in enumerate(tbl):
        if 0 <= col_id < len(row):
            cell = "" if row[col_id] is None else str(row[col_id])
            cell_toks = set(cleaner(cell, create_ngrams, ngram_size))
            if toks.issubset(cell_toks):
                rows.add(r)
    return rows

def candidate_cols_for_values(table_list, projections, values, tau, create_ngrams=False, ngram_size=2):
    hits_by_table_col = defaultdict(set)  # (tid, cid) -> set(values_matched)
    max_tid = len(table_list) - 1
    for v in values:
        for (tid, rid, cid) in _value_hits(projections, v, create_ngrams, ngram_size):
            if not (0 <= tid <= max_tid):
                continue
            tbl = table_list[tid]
            if not (0 <= rid < len(tbl)):
                continue
            row = tbl[rid]
            if not (0 <= cid < len(row)):
                continue
            hits_by_table_col[(tid, cid)].add(v)
    out = defaultdict(set)  # tid -> {cid,...}
    for (tid, cid), matched in hits_by_table_col.items():
        if len(matched) >= tau:
            out[tid].add(cid)
    return out


In [ ]:
# --- Patch: stronger value lookup for III-C when numbers weren't indexed ---

def _value_hits(projections, value, create_ngrams=False, ngram_size=2):
    toks = cleaner(value, create_ngrams, ngram_size)
    postings = set()
    for t in toks:
        postings |= projections.get(t, set())

    if postings:
        return postings

    # Fallback: numeric (or mixed) values often weren't token-indexed by A/B.
    needle = str(value).strip()
    if needle and any(ch.isdigit() for ch in needle):
        for tid, tbl in enumerate(table_list):
            for rid, row in enumerate(tbl):
                for cid, cell in enumerate(row):
                    if cell is None:
                        continue
                    if needle in str(cell):
                        postings.add((tid, rid, cid))
    return postings


In [39]:
# --- III-C (3/4): Section III-C core (indirect transformations via joins) ---

from collections import defaultdict, deque

def fd_holds_X_to_Z_on_examples(table_list, table_id, x_col, z_col, E_x_to_y, tau, create_ngrams=False, ngram_size=2):
    ok = 0
    seen_contradiction = False
    for x_val, _ in E_x_to_y:
        x_rows = rows_containing_value_in_col(table_list, table_id, x_col, x_val, create_ngrams, ngram_size)
        if not x_rows:
            continue
        z_vals = set()
        for r in x_rows:
            row = table_list[table_id][r]
            if 0 <= z_col < len(row):
                z_vals.add(str(row[z_col]))
        if len(z_vals) == 1:
            ok += 1
        elif len(z_vals) > 1:
            seen_contradiction = True
    return (ok >= tau) and (not seen_contradiction)

def fd_holds_Z_to_Y_on_examples(table_list, table_id, z_col, y_col, E_x_to_y, tau, create_ngrams=False, ngram_size=2):
    ok = 0
    seen_contradiction = False
    # We verify that, for at least tau examples, all matching rows agree on Y (unique Y per Z).
    for _, y_val in E_x_to_y:
        # here we don't have the exact Z per example; we conservatively check consistency where y tokens match
        y_tokens = set(cleaner(str(y_val), create_ngrams, ngram_size))
        if not y_tokens:
            continue
        y_vals_found = set()
        tbl = table_list[table_id]
        for r, row in enumerate(tbl):
            if 0 <= z_col < len(row) and 0 <= y_col < len(row):
                y_cell = "" if row[y_col] is None else str(row[y_col])
                if y_tokens.issubset(set(cleaner(y_cell, create_ngrams, ngram_size))):
                    y_vals_found.add(y_cell)
        if len(y_vals_found) == 1:
            ok += 1
        elif len(y_vals_found) > 1:
            seen_contradiction = True
    return (ok >= tau) and (not seen_contradiction)

def extract_Z_set(table_list, table_id, x_col, z_col, X_values, create_ngrams=False, ngram_size=2):
    Z = set()
    lineage = defaultdict(set)  # x -> {z,...}
    for x_val in X_values:
        x_rows = rows_containing_value_in_col(table_list, table_id, x_col, x_val, create_ngrams, ngram_size)
        for r in x_rows:
            row = table_list[table_id][r]
            if 0 <= z_col < len(row):
                z = str(row[z_col])
                Z.add(z)
                lineage[str(x_val)].add(z)
    return Z, dict(lineage)

def find_join_columns(table_list, projections, E, tau=2, create_ngrams=False, ngram_size=2):
    """
    Find, per table, columns (x_col, z_col) where X -> Z holds on >= tau examples.
    Returns: {table_id: [ {x_col, z_col, Z, x_to_z_lineage}, ... ]}
    """
    E_x_to_y = [(str(x), str(y)) for (x, y) in E]
    X_values = {x for x, _ in E_x_to_y}

    x_cols_by_table = candidate_cols_for_values(table_list, projections, X_values, tau, create_ngrams, ngram_size)
    results = defaultdict(list)

    for tid, x_cols in x_cols_by_table.items():
        if not (0 <= tid < len(table_list)):
            continue
        tbl = table_list[tid]
        num_cols = max((len(r) for r in tbl), default=0)
        for x_col in x_cols:
            for z_col in range(num_cols):
                if z_col == x_col:
                    continue
                if fd_holds_X_to_Z_on_examples(table_list, tid, x_col, z_col, E_x_to_y, tau, create_ngrams, ngram_size):
                    Z, x2z = extract_Z_set(table_list, tid, x_col, z_col, X_values, create_ngrams, ngram_size)
                    # cardinality sanity check: Z should be large enough to be useful
                    if len(Z) >= len({y for _, y in E_x_to_y}):
                        results[tid].append({
                            "x_col": x_col,
                            "z_col": z_col,
                            "Z": Z,
                            "x_to_z_lineage": x2z,
                        })
    return results

def find_joinable_tables(table_list, projections, Z_set, Y_values, tau=2, create_ngrams=False, ngram_size=2):
    """
    Given candidate Z values, find tables having a Z column (>= tau matches) and a Y column with Z -> Y (>= tau).
    """
    z_cols_by_table = candidate_cols_for_values(table_list, projections, Z_set, tau, create_ngrams, ngram_size)
    joined = []
    E_dummy = [(None, str(y)) for y in Y_values]  # only Y is used in the Z->Y check

    for tid, z_cols in z_cols_by_table.items():
        if not (0 <= tid < len(table_list)):
            continue
        tbl = table_list[tid]
        num_cols = max((len(r) for r in tbl), default=0)
        for z_col in z_cols:
            # support: how many Z from Z_set actually appear in this column
            matched_Z = set()
            for z in Z_set:
                if rows_containing_value_in_col(table_list, tid, z_col, z, create_ngrams, ngram_size):
                    matched_Z.add(z)
            for y_col in range(num_cols):
                if y_col == z_col:
                    continue
                if fd_holds_Z_to_Y_on_examples(table_list, tid, z_col, y_col, E_dummy, tau, create_ngrams, ngram_size):
                    joined.append({
                        "table_id": tid,
                        "z_col": z_col,
                        "y_col": y_col,
                        "support": len(matched_Z),
                        "Z_in_col": matched_Z,
                    })
    return joined

def table_joiner_bfs(table_list, projections, E, Q, tau=2, max_path_len=1, create_ngrams=False, ngram_size=2):
    """
    Discover indirect X -> Z -> Y transformations satisfying FD constraints on >= tau examples.
    """
    E_x_to_y = [(str(x), str(y)) for (x, y) in E]
    X_values = {x for x, _ in E_x_to_y}
    Y_values = {y for _, y in E_x_to_y}

    # Step 1: discover (T, x_col, z_col) with X -> Z
    x_to_z_candidates = find_join_columns(table_list, projections, E_x_to_y, tau, create_ngrams, ngram_size)

    results = {"paths": [], "joined_tables": []}

    frontier = deque()
    for tid, cand_list in x_to_z_candidates.items():
        for c in cand_list:
            frontier.append({
                "path": [{"table_id": tid, "x_col": c["x_col"], "z_col": c["z_col"], "Z": c["Z"]}],
                "Z": c["Z"],
                "depth": 1
            })

    visited_Z_snapshots = set()
    def z_sig(Z):
        # short signature to avoid cycles
        return tuple(sorted(list(Z))[:64])

    while frontier:
        node = frontier.popleft()
        Zcur = node["Z"]
        sig = z_sig(Zcur)
        if sig in visited_Z_snapshots:
            continue
        visited_Z_snapshots.add(sig)

        # Step 2: find tables with Z -> Y
        joined = find_joinable_tables(table_list, projections, Zcur, Y_values, tau, create_ngrams, ngram_size)
        results["joined_tables"].extend(joined)

        # Project Y candidates for query Xs through any matched Z rows (liberal projection)
        produced = defaultdict(set)
        for j in joined:
            tid = j["table_id"]; zc = j["z_col"]; yc = j["y_col"]
            tbl = table_list[tid]
            for r, row in enumerate(tbl):
                if 0 <= zc < len(row) and 0 <= yc < len(row):
                    z_val = str(row[zc]); y_val = str(row[yc])
                    z_toks = set(cleaner(z_val, create_ngrams, ngram_size))
                    for xq in Q:
                        if set(cleaner(str(xq), create_ngrams, ngram_size)).issubset(z_toks):
                            produced[str(xq)].add(y_val)

        results["paths"].append({
            "path_len": 1,
            "via": node["path"],
            "covers_tau_examples": True,   # ensured by FD checks
            "produced_y_for_q": {k: sorted(list(v)) for k, v in produced.items()}
        })

        # Step 3: optionally expand to longer chains (X -> Z1 -> Z2 -> ... -> Y)
        if node["depth"] < max_path_len:
            # Treat current Z as the new X to find Z' with Z -> Z'
            E_proxy = [(z, z) for z in Zcur]
            z_to_zprime = find_join_columns(table_list, projections, E_proxy, tau, create_ngrams, ngram_size)
            for tid2, cand_list2 in z_to_zprime.items():
                for c2 in cand_list2:
                    frontier.append({
                        "path": node["path"] + [{"table_id": tid2, "x_col": c2["x_col"], "z_col": c2["z_col"], "Z": c2["Z"]}],
                        "Z": c2["Z"],
                        "depth": node["depth"] + 1
                    })

    return results


In [40]:
# --- III-C (4/4): example call (adjust E/Q and tau to your data) ---

E = [('1929', 'Robert Crawford')]  # your example pairs
Q = ['1929']                       # Xs to transform

out = table_joiner_bfs(
    table_list, projections, E, Q,
    tau=1,            # use 2+ if you have multiple pairs
    max_path_len=1,   # set >1 to allow longer X->Z1->Z2->Y chains
    create_ngrams=False,
    ngram_size=2
)
out


{'paths': [], 'joined_tables': []}

In [ ]:
# --- III-C DEBUG (A): where do X-values appear? ---

from collections import defaultdict
import itertools

def inspect_x_columns(table_list, projections, X_values, max_samples=5):
    by_table_col = defaultdict(set)  # (tid, cid) -> set(X matched)

    for x in X_values:
        for (tid, rid, cid) in _value_hits(projections, x):
            # guards already inside _value_hits/purge, but we keep it safe
            if not (0 <= tid < len(table_list)): 
                continue
            row = table_list[tid][rid]
            if not (0 <= cid < len(row)):
                continue
            by_table_col[(tid, cid)].add(x)

    # Print summary
    if not by_table_col:
        print("No X hits found in any table/column.")
        return {}

    print("X appears in the following (table_id, col_id) with counts:")
    for (tid, cid), xs in sorted(by_table_col.items(), key=lambda kv: (-len(kv[1]), kv[0][0], kv[0][1]))[:20]:
        print(f"  T{tid} C{cid}: matched {len(xs)} of {len(X_values)} X values")

    # Sample a few rows from each interesting column
    print("\nSamples (first few rows) from those columns:")
    for (tid, cid), xs in itertools.islice(sorted(by_table_col.items(), key=lambda kv: (-len(kv[1]), kv[0][0], kv[0][1])), 5):
        print(f"\n=== T{tid} C{cid} (matches {len(xs)} Xs) ===")
        tbl = table_list[tid]
        shown = 0
        for r, row in enumerate(tbl):
            if cid < len(row):
                val = row[cid]
                if val is None:
                    continue
                text = str(val)
                # show rows that contain any X substring
                if any(str(x) in text for x in X_values):
                    # print the whole row to understand context
                    print(f"r={r}: {row}")
                    shown += 1
                    if shown >= max_samples:
                        break
    return by_table_col

inspect_x_columns(table_list, projections, X_values=['1929'])


X appears in the following (table_id, col_id) with counts:
  T5 C0: matched 1 of 1 X values

Samples (first few rows) from those columns:

=== T5 C0 (matches 1 Xs) ===
r=11: ['1929–30', 'Hancock Hockey Club']


defaultdict(set, {(5, 0): {'1929'}})

In [44]:
# --- III-C DEBUG (B): relaxed FD options ---

from collections import Counter, defaultdict

def fd_X_to_Z(table_list, tid, x_col, z_col, E_x_to_y, tau, mode='strict', tol=0, create_ngrams=False, ngram_size=2):
    ok = 0
    for x_val, _ in E_x_to_y:
        rows = rows_containing_value_in_col(table_list, tid, x_col, x_val, create_ngrams, ngram_size)
        if not rows:
            continue
        z_vals = [str(table_list[tid][r][z_col]) for r in rows if 0 <= z_col < len(table_list[tid][r])]
        z_vals = [z for z in z_vals if z is not None]
        if not z_vals:
            continue

        if mode == 'strict':
            if len(set(z_vals)) == 1:
                ok += 1
            else:
                return False  # any contradiction kills strict
        elif mode == 'majority':
            c = Counter(z_vals)
            maj = c.most_common(1)[0][1]
            # tolerate up to 'tol' off-majority appearances
            if maj >= len(z_vals) - tol:
                ok += 1
        elif mode == 'exists':
            ok += 1
        else:
            raise ValueError("mode must be 'strict', 'majority', or 'exists'")
    return ok >= tau

def fd_Z_to_Y(table_list, tid, z_col, y_col, E_x_to_y, tau, mode='strict', tol=0, create_ngrams=False, ngram_size=2):
    ok = 0
    # We only have Y side in examples; we check that rows agreeing on Y are not contradictory for ≥ tau examples.
    for _, y_val in E_x_to_y:
        y_toks = set(cleaner(str(y_val), create_ngrams, ngram_size))
        if not y_toks:
            continue
        y_vals_found = []
        tbl = table_list[tid]
        for r, row in enumerate(tbl):
            if 0 <= z_col < len(row) and 0 <= y_col < len(row):
                y_cell = "" if row[y_col] is None else str(row[y_col])
                if y_toks.issubset(set(cleaner(y_cell, create_ngrams, ngram_size))):
                    y_vals_found.append(y_cell)
        if not y_vals_found:
            continue
        if mode == 'strict':
            if len(set(y_vals_found)) == 1:
                ok += 1
            else:
                return False
        elif mode == 'majority':
            c = Counter(y_vals_found)
            maj = c.most_common(1)[0][1]
            if maj >= len(y_vals_found) - tol:
                ok += 1
        elif mode == 'exists':
            ok += 1
    return ok >= tau

# Wrappers that use relaxed FDs
def find_join_columns_relaxed(table_list, projections, E, tau=2, mode='strict', tol=0, create_ngrams=False, ngram_size=2):
    E_x_to_y = [(str(x), str(y)) for (x, y) in E]
    X_values = {x for x, _ in E_x_to_y}
    x_cols_by_table = candidate_cols_for_values(table_list, projections, X_values, tau, create_ngrams, ngram_size)
    results = defaultdict(list)
    for tid, x_cols in x_cols_by_table.items():
        if not (0 <= tid < len(table_list)): 
            continue
        tbl = table_list[tid]
        num_cols = max((len(r) for r in tbl), default=0)
        for x_col in x_cols:
            for z_col in range(num_cols):
                if z_col == x_col:
                    continue
                if fd_X_to_Z(table_list, tid, x_col, z_col, E_x_to_y, tau, mode, tol, create_ngrams, ngram_size):
                    # collect Z set (same as before)
                    Z = set()
                    for x_val, _ in E_x_to_y:
                        rows = rows_containing_value_in_col(table_list, tid, x_col, x_val, create_ngrams, ngram_size)
                        for r in rows:
                            row = table_list[tid][r]
                            if 0 <= z_col < len(row):
                                Z.add(str(row[z_col]))
                    if Z:
                        results[tid].append({"x_col": x_col, "z_col": z_col, "Z": Z})
    return results

def find_joinable_tables_relaxed(table_list, projections, Z_set, Y_values, tau=2, mode='strict', tol=0, create_ngrams=False, ngram_size=2):
    z_cols_by_table = candidate_cols_for_values(table_list, projections, Z_set, tau, create_ngrams, ngram_size)
    joined = []
    E_dummy = [(None, str(y)) for y in Y_values]
    for tid, z_cols in z_cols_by_table.items():
        if not (0 <= tid < len(table_list)):
            continue
        tbl = table_list[tid]
        num_cols = max((len(r) for r in tbl), default=0)
        for z_col in z_cols:
            for y_col in range(num_cols):
                if y_col == z_col:
                    continue
                if fd_Z_to_Y(table_list, tid, z_col, y_col, E_dummy, tau, mode, tol, create_ngrams, ngram_size):
                    joined.append({"table_id": tid, "z_col": z_col, "y_col": y_col})
    return joined

def table_joiner_bfs_relaxed(table_list, projections, E, Q, tau=1, mode='majority', tol=1, create_ngrams=False, ngram_size=2):
    E_x_to_y = [(str(x), str(y)) for (x, y) in E]
    X_values = {x for x, _ in E_x_to_y}
    Y_values = {y for _, y in E_x_to_y}
    x2z = find_join_columns_relaxed(table_list, projections, E_x_to_y, tau, mode, tol, create_ngrams, ngram_size)
    results = {"paths": [], "joined_tables": []}
    for tid, lst in x2z.items():
        for c in lst:
            joined = find_joinable_tables_relaxed(table_list, projections, c["Z"], Y_values, tau, mode, tol, create_ngrams, ngram_size)
            results["joined_tables"].extend(joined)
            # simple projection like before
            produced = defaultdict(set)
            for j in joined:
                t2 = j["table_id"]; zc = j["z_col"]; yc = j["y_col"]
                for r, row in enumerate(table_list[t2]):
                    if 0 <= zc < len(row) and 0 <= yc < len(row):
                        z_val = str(row[zc]); y_val = str(row[yc])
                        z_toks = set(cleaner(z_val, create_ngrams, ngram_size))
                        for xq in Q:
                            if set(cleaner(str(xq), create_ngrams, ngram_size)).issubset(z_toks):
                                produced[str(xq)].add(y_val)
            results["paths"].append({
                "path_len": 1,
                "via": [{"table_id": tid, "x_col": c["x_col"], "z_col": c["z_col"]}],
                "produced_y_for_q": {k: sorted(list(v)) for k, v in produced.items()}
            })
    return results

# Try relaxed:
E = [('1929', 'Robert Crawford')]
Q = ['1929']
rel = table_joiner_bfs_relaxed(table_list, projections, E, Q, tau=1, mode='exists')
print(rel)


{'paths': [], 'joined_tables': []}
